In [1]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
from collections import Counter

In [3]:
import numpy as np

In [4]:
from bs4 import BeautifulSoup
import re

In [5]:
import requests
from pprint import pprint

In [6]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from tqdm import tqdm
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/arinkazam/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/arinkazam/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
from fake_useragent import UserAgent

In [8]:
from scipy import stats as st

In [9]:
from pymorphy2 import MorphAnalyzer

In [10]:
import pandas as pd

In [11]:
# имитируем пользователя и скачиваем основную страницу со списками отелей с сайта 101hotels.com
ua = UserAgent()
headers = {'User-Agent': ua.random}
session = requests.session()
response = session.get('https://m.101hotels.com/main/cities/moskva/inexpensive', headers=headers)
page = response.text
soup = BeautifulSoup(page, 'html.parser')
looking_for_reviews = soup.find_all('div', {'class': 'hotel__title pr-5'})

In [12]:
# скачиваем отзывы, размеченные как позитивные, они выведены на страницах отелей в разделе "что гостям понравилось"
reviews_positive = []
for item in looking_for_reviews:
    link = item.find('a').attrs['href']
    full_link = 'https://m.101hotels.com' + link
    fragm = session.get(full_link, headers=headers).text
    soup_fragm = BeautifulSoup(fragm, 'html.parser')
    reviews_fragm = soup_fragm.find_all('div', {'class': 'hotel-top-reviews__text'})
    for elem in reviews_fragm:
        review = elem.find('div', {'class': 'response__text-full'}).text
        reviews_positive.append(review)

In [13]:
# скачиваем негативные отзывы, их можно найти в разделе всех отзывов под меткой "response__text response__text_dislike"
reviews_negative = []
for item in looking_for_reviews:
    link = item.find('a').attrs['href']
    full_link = 'https://m.101hotels.com' + link
    fragm = session.get(full_link, headers=headers).text
    soup_fragm = BeautifulSoup(fragm, 'html.parser')
    reviews_link = 'https://m.101hotels.com' + soup_fragm.find('div', {'class': 'hotel-rating-item__text'}).find('a').attrs['href']
    reviews_page = session.get(reviews_link, headers=headers).text
    reviews_content = BeautifulSoup(reviews_page, 'html.parser')
    negative_options = reviews_content.select_one('div', {'class': 'response__content'}).find_all('div', {'class': 'response__text response__text_dislike'})
    for i in negative_options:
        reviews_negative.append(i.text)  

In [14]:
# создаем баланс классов
if len(reviews_negative) > len(reviews_positive):
    reviews_negative = reviews_negative[:len(reviews_positive)]
else:
    reviews_positive = reviews_positive[:len(reviews_negative)]

In [15]:
# создаем датасет с отзывами
data = {}
data['text'] = []
data['target'] = []
for post in reviews_positive:
    data['text'].append(post)
    data['target'].append(1)
for post_neg in reviews_negative:
    data['text'].append(post_neg)
    data['target'].append(0)
df = pd.DataFrame(data)
df = shuffle(df)
df.reset_index(drop=True, inplace=True)

In [16]:
# выводим пример того, как выглядит датасет
df

,text,target
0,"Мягкий очень матрас, на любителя, нет ведра и ...",0
1,"От метро близко, но будьте готовы идти по темн...",0
2,"Очень уставшие номера морально и физически, но...",1
3,"Чисто,аккуратно,в номера есть все необходимое,...",1
4,"Красивое здание, красивые стены и двери, пол к...",1
...,...,...
243,Уже не первый раз останавливаюсь в этой гостин...,1
244,"Хотелось бы, чтоб отели уходили от ковровых на...",0
245,"Понравился вежливый персонал, внутренний социу...",1
246,Отличное расположение-близость метро и большог...,1


In [17]:
# сохраняем датасет в файл на будущее
df.to_csv('pos_neg_posts_df.csv')

In [18]:
morph = MorphAnalyzer()
stop_w = set(stopwords.words('russian'))

In [19]:
# создаем функцию для препроцессинга отзывов
def preprocessing(arr):
    corpus=[]
    for text in tqdm(arr):
        text = text.replace('\r\n', ' ')
        words=[word.lower() for word in word_tokenize(text) if ((word.isalpha() == 1) & (word not in stop_w))]
        lemmatized_output = ' '.join([morph.parse(w)[0].normal_form for w in words])
        corpus.append(lemmatized_output)
    return corpus

In [20]:
# применяем функцию к датасету, разбиваем датасет на признаки и целевую переменную
df['text'] = preprocessing(df['text'])
df_x = df['text']
df_y = df['target']

100%|████████████████████████████████████████████████████████████████████████████████| 248/248 [00:02<00:00, 109.15it/s]


In [21]:
# разбиваем датасет на тренировочную и тестовую выборку
np.random.seed(11)
X_train, X_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.3, random_state=10)

In [22]:
# делаем список только позитивных и только негативных отзывов из датафрейма, смотрим пересечение, смотрим частотность
reviews_positive_train = list(X_train[y_train == 1])
reviews_negative_train = list(X_train[y_train == 0])

positive_words = (' '.join(reviews_positive_train)).split()
dict_positive_apppear = Counter(positive_words)
negative_words = (' '.join(reviews_negative_train)).split()
dict_negative_apppear = Counter(negative_words)

set_positive_words = set(positive_words)
set_negative_words = set(negative_words)
in_both_types = set_positive_words & set_negative_words

list_positive_words = list(set_positive_words)
list_negative_words = list(set_negative_words)
list_in_both_types = list(in_both_types)

In [23]:
# убираем пересечение слов негативных и позитивных отзывов из обоих списков
i = 0
j = 0
while i < len(list_positive_words):
    if list_positive_words[i] in list_in_both_types:
        list_positive_words.pop(i)
    else:
        i += 1

while j < len(list_negative_words):
    if list_negative_words[j] in list_in_both_types:
        list_negative_words.pop(j)
    else:
        j += 1
        

In [24]:
# пробуем учесть частотности слов и их длину (так могут исключиться вещи, которые не вошли в стоп-слова, но могут ими считаться)
t = 0
k = 0
while t < len(list_positive_words):
    if dict_positive_apppear[list_positive_words[t]] < 3 or len(list_positive_words[t]) < 3:
        list_positive_words.pop(t)
    else:
        t += 1

while k < len(list_negative_words):
    if dict_negative_apppear[list_negative_words[k]] < 3 or len(list_negative_words[k]) < 3:
        list_negative_words.pop(k)
    else:
        k += 1

In [25]:
# создаем итоговые множества позитивных и негативных слов из отзывов
only_positive = set(list_positive_words)
only_negative = set(list_negative_words)

In [26]:
# создаем функцию классификации отзывов по имеющимся множествам позитивных и негативных слов из отзывов 
def predict_tone(arr, pos_l, neg_l):
    ans = []
    for text in arr:
        content = set(text.split())
        # считаем, каких вхождений больше, выводим превалирующее
        if len(content & pos_l) > len(content & neg_l):
            ans.append(1)
        else:
            ans.append(0)
    return ans

In [27]:
# создаем функцию для подсчета accuracy
def accuracy_score(prediction, y):
    return (len(prediction) - np.sum(np.absolute(np.array(prediction) - np.array(y)))) / len(prediction)

In [28]:
# делаем предсказание на тренировочной и тестовых выборках
predicted_train = predict_tone(X_train, only_positive, only_negative)
predicted_test = predict_tone(X_test, only_positive, only_negative)

In [29]:
# выводим accuracy для тренировочной и тестовой выборок
print(f"train = {accuracy_score(predicted_train, y_train)}")
print(f"test = {accuracy_score(predicted_test, y_test)}")

train = 0.9826589595375722
test = 0.8266666666666667


#### Как улучшить алгоритм:

1. попробуем использовать биграммы вместо одиночных слов
   
2. векторизация текста и подсчет евклидовой метрики между векторами тренировочной и тестовой выборок

In [30]:
# реализация первого способа, функция для поиска биграмм позитивных и негативных отзывов
def bigrams_search(arr):
    bigrams = []
    for text in tqdm(arr):
        text_new = re.findall(r'[^\s]+[\s][^\s]+', text)        
        for bigram in text_new:
            bigrams.append(bigram)
        text_add = ' '.join(text.split()[1:])
        text_add_new = re.findall(r'[^\s]+[\s][^\s]+', text_add)
        for bigram_add in text_add_new:
            bigrams.append(bigram_add)
    return bigrams

In [31]:
bigrams_positive = bigrams_search(X_train[y_train == 1])
bigrams_negative = bigrams_search(X_train[y_train == 0])

100%|████████████████████████████████████████████████████████████████████████████████| 86/86 [00:00<00:00, 26721.25it/s]


In [32]:
# смотрим на частотность биграмм, создаем список биграмм позитивных и негативных отзывов, смотрим пересечение
dict_positive_bigr = Counter(bigrams_positive)
dict_negative_bigr = Counter(bigrams_negative)

set_positive_bigr = set(bigrams_positive)
set_negative_bigr = set(bigrams_negative)
in_both_bigr = set_positive_bigr & set_negative_bigr

list_positive_bigr = list(set_positive_bigr)
list_negative_bigr = list(set_negative_bigr)
list_in_both_bigr = list(in_both_bigr)

In [33]:
#  убираем пересечение из списков, чистим списки по частотности
f = 0
l = 0
while f < len(list_positive_bigr):
    if list_positive_bigr[f] in list_in_both_bigr or dict_positive_bigr[list_positive_bigr[f]] < 3:
        list_positive_bigr.pop(f)
    else:
        f += 1

while l < len(list_negative_bigr):
    if list_negative_bigr[l] in list_in_both_bigr or dict_negative_bigr[list_negative_bigr[l]] < 3:
        list_negative_bigr.pop(l)
    else:
        l += 1

In [34]:
only_positive_bigr = set(list_positive_bigr)
only_negative_bigr = set(list_negative_bigr)

In [35]:
# функция для предсказания тональности по биграммам
def predict_tone_bigr(arr, pos_l, neg_l):
    ans = []
    for text in arr:
        content = set(re.findall(r'[^\s]+[\s][^\s]+', text)) | set(re.findall(r'[^\s]+[\s][^\s]+', ' '.join(text.split()[1:])))
        if len(content & pos_l) > len(content & neg_l):
            ans.append(1)
        else:
            ans.append(0)
    return ans

In [36]:
# предсказания на тренировочной и тестовой выборках
predicted_train_bigr = predict_tone_bigr(X_train, only_positive_bigr, only_negative_bigr)
predicted_test_bigr = predict_tone_bigr(X_test, only_positive_bigr, only_negative_bigr)

In [37]:
# выводим accuracy для тренировочной и тестовой выборок
print(f"train = {accuracy_score(predicted_train_bigr, y_train)}")
print(f"test = {accuracy_score(predicted_test_bigr, y_test)}")

train = 0.7976878612716763
test = 0.64


In [38]:
# реализация второго способа
vect = CountVectorizer(min_df = 10)
X_train_bow = vect.fit_transform(X_train).toarray()
X_test_bow = vect.transform(X_test).toarray()

In [39]:
# функция для предсказания целевой переменной по векторам
def predict_new_way(k, y_train, X_train, X_test):
    # сортирую полученные значения, выводя их индексы, оставляю k нужных
    # здесь я с помощью евклидовой метрики вычисляю расстояние между объектами тестовой и тренировочной выборки
    # потом сортирую полученные значения, выводя их индексы, оставляю k нужных
    index = np.argsort(((((X_test[:, np.newaxis, :] - X_train[np.newaxis, :, :])**2).sum(axis=2))**0.5), axis = -1)[:, :k]
    # вывожу моду среди значений по индексам
    return st.mode(np.array(y_train)[index], axis=1).mode

In [40]:
# предсказание на тестовой выборке
predicted_new_way = predict_new_way(3, y_train, X_train_bow, X_test_bow)

In [41]:
# выводим accuracy для тестовой выборки
print(accuracy_score(predicted_new_way, y_test))

0.7066666666666667
